In [1]:
import requests
import lxml.html as lh
import pandas as pd
import re 

In [2]:
url = 'https://en.wikipedia.org/wiki/Bernd_Leno'

def getNationalTeamTable(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    tables = doc.xpath('//table')
    for table in tables:
        captions = table.xpath('//caption')
        for caption in captions:
            text = caption.text_content()
            if "Appearances and goals by national team and year" in text:
                return table

# TODO: Better Look Up
def getAppsForPlayer(wiki_name):
    url = 'https://en.wikipedia.org/wiki/'+wiki_name
    table = getNationalTeamTable(url)
    if table:
        rows = table.xpath('//tr')
        i=0
        for row in rows:
            i=i+1
            if "Total" in row.text_content():
                if len(row) == 3:
                    total = row[1]
                    return int(total.text_content()), url
    print('Error Getting Apps for: ' + url)
    return -1 ,url

In [3]:
def getPlayerRows(url):
    playerRows = []
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    rows = doc.xpath('//tr')
    for row in rows:
        if len(row) == 12:
            name_cell = row[3]
            nationality_cell = row[2]
            text = name_cell.text_content().strip()
            skip = text.isnumeric() or len(text) == 0 or text == 'Name'
            if not skip:
                playerRows.append((name_cell,nationality_cell))
    if len(playerRows) == 0:
        print('Error Getting Players from: ' + url)
    else:
        print('Jheeeze found {} players'.format(len(playerRows)))
    return playerRows
    
    
def getWikiLink(cell):
    links = list(cell.iterlinks())
    if len(links)==0:
        return 'ERROR'
    return links[0][2].replace('/wiki/','')


def getWikiDetails(cells):
    player = cells[0]
    country_cell = cells[1]
    display_name = player.text_content()
#     links = list(player.iterlinks())
#     wiki_name = links[0][2].replace('/wiki/','')
    return display_name, getWikiLink(player), getWikiLink(country_cell)

def addPlayerApps(name_tuple):
    display_name = name_tuple[0]
    wiki_name = name_tuple[1]
    country = name_tuple[2]
    (apps,url) = getAppsForPlayer(wiki_name)
    return display_name, wiki_name, country, apps, url

In [4]:
def addPlayerApps(name_tuple):
    display_name = name_tuple[0]
    wiki_name = name_tuple[1]
    country = name_tuple[2]
    (apps,url) = getAppsForPlayer(wiki_name)
    return display_name, wiki_name, country, apps, url

In [5]:
def getAppearanceArray(url):
    playerRows = getPlayerRows(url)
    linksArray = [getWikiDetails(player) for player in playerRows]
    appsArray = [addPlayerApps(names) for names in linksArray]
    return appsArray

In [6]:
def getTeams(url):
    teams = []
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    rows = doc.xpath('//tr')
    for row in rows:
        if len(row) == 4:
            cell = row[0]
            text = cell.text_content().strip()
            skip = text == 'Team' or text == 'Rank' or text.isnumeric()
            if not skip:
                teams.append(cell)
    return teams


def addTeamToEachItem(player,teamName,squad_url):
    (display_name, wiki_name, country, apps, url) = player
    return teamName,squad_url,display_name, wiki_name, country, apps, url

In [7]:
def fuckedIt(name):
    return 'Fucked It','should be wiki link for'+name,'but Im weird', None, None, None, None

In [8]:
def getDataFrameForLeague(url):
    teams = getTeams(url)
    output = []
    for team in teams:
        teamDisplayName = team.text_content()
        wiki_name = getWikiLink(team)
        if wiki_name == 'ERROR':
            output = output.append(fuckedIt(teamDisplayName))
        else:
            squad_url = 'https://en.wikipedia.org/wiki/2019-20_'+wiki_name+'_season'
            print('Doing Look up for: '+ teamDisplayName)
            appearanceArray = getAppearanceArray(squad_url)
            updatedArray = [addTeamToEachItem(player,teamDisplayName,squad_url) for player in appearanceArray]
            output = output + updatedArray
    return pd.DataFrame(output, columns = ['teamName','squad_url','display_name', 'wiki_name', 'country', 'apps', 'url']) 

In [9]:
url = 'https://en.wikipedia.org/wiki/2020%E2%80%9321_Premier_League#Teams'
data = getDataFrameForLeague(url)
print(len(data))

Doing Look up for: Arsenal

Jheeeze found 32 players


<ipython-input-2-60738d804e64>:18: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if table:


Error Getting Apps for: https://en.wikipedia.org/wiki/Kieran_Tierney
Error Getting Apps for: https://en.wikipedia.org/wiki/Lucas_Torreira
Error Getting Apps for: https://en.wikipedia.org/wiki/Rob_Holding
Error Getting Apps for: https://en.wikipedia.org/wiki/Pablo_Mar%C3%AD
Error Getting Apps for: https://en.wikipedia.org/wiki/Reiss_Nelson
Error Getting Apps for: https://en.wikipedia.org/wiki/Emiliano_Mart%C3%ADnez
Error Getting Apps for: https://en.wikipedia.org/wiki/Konstantinos_Mavropanos
Error Getting Apps for: https://en.wikipedia.org/wiki/Joe_Willock
Error Getting Apps for: https://en.wikipedia.org/wiki/Matteo_Guendouzi
Error Getting Apps for: https://en.wikipedia.org/wiki/Eddie_Nketiah
Error Getting Apps for: https://en.wikipedia.org/wiki/Emile_Smith_Rowe
Error Getting Apps for: https://en.wikipedia.org/wiki/Matt_Macey
Error Getting Apps for: https://en.wikipedia.org/wiki/Gabriel_Martinelli
Error Getting Apps for: https://en.wikipedia.org/wiki/William_Saliba
Doing Look up for: As

Error Getting Apps for: https://en.wikipedia.org/wiki/%C3%87a%C4%9Flar_S%C3%B6y%C3%BCnc%C3%BC
Error Getting Apps for: https://en.wikipedia.org/wiki/Jonny_Evans
Error Getting Apps for: https://en.wikipedia.org/wiki/Demarai_Gray
Error Getting Apps for: https://en.wikipedia.org/wiki/Marc_Albrighton
Error Getting Apps for: https://en.wikipedia.org/wiki/Islam_Slimani
Error Getting Apps for: https://en.wikipedia.org/wiki/Filip_Benkovi%C4%87
Error Getting Apps for: https://en.wikipedia.org/wiki/Ayoze_P%C3%A9rez
Error Getting Apps for: https://en.wikipedia.org/wiki/Hamza_Choudhury
Error Getting Apps for: https://en.wikipedia.org/wiki/Ricardo_Pereira_(footballer,_born_1993)
Error Getting Apps for: https://en.wikipedia.org/wiki/Matty_James
Error Getting Apps for: https://en.wikipedia.org/wiki/Adrien_Silva
Error Getting Apps for: https://en.wikipedia.org/wiki/Nampalys_Mendy
Error Getting Apps for: https://en.wikipedia.org/wiki/Dennis_Praet
Error Getting Apps for: https://en.wikipedia.org/wiki/Fou

In [10]:
def writeDataToFile(df,fileName):
    df.to_csv(fileName,index=False)
    print(df.head())

In [11]:
# writeDataToFile(data,"PremSample.csv")

In [12]:
import csv

with open('leagues.csv', newline='') as csvfile:
    leagueList = list(csv.reader(csvfile))

print(leagueList)

FileNotFoundError: [Errno 2] No such file or directory: 'leagues.csv'